In [1]:
import h5py
import numpy as np
import torchvision.transforms.v2 as transforms
from scipy.signal import fftconvolve
import torch
from tqdm.auto import tqdm

from pathlib import Path

import shutil

from vacation.data import download_dataset

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

rng = np.random.default_rng(42)

In [3]:
try:
    download_dataset("../../.data/", overwrite=False, redownload=False)
except FileExistsError:
    pass

/home/jovyan/vacation/vacation/data/dataloader.py:26: UserWarning: The data file already exists at this location!Use a different location or set 'redownload=True'.
  warnings.warn(


Checksum passed successfully.


In [4]:
def numpy_to_tensor(image):
    return torch.from_numpy(image).permute(2, 0, 1).float().to("cuda")


def tensor_to_numpy(im_tensor):
    im_tensor = torch.clamp(im_tensor, 0, 1)
    return im_tensor.permute(1, 2, 0).cpu().numpy()


transform = transforms.Compose(
    [
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.GaussianNoise(mean=0.0, sigma=0.1, clip=True),
        transforms.ColorJitter(brightness=0.4, contrast=0, saturation=0.5, hue=0),
    ]
)


def augmented_class(path: str, class_index: int, target_count: int = 2600):

    with h5py.File(path, "r") as hf:

        labels = np.array(hf["ans"])
        images = hf["images"]

        original_indices = np.where(labels == class_index)[0]
        needed_count = target_count - len(original_indices)

        print(
            f"[Class {class_index}] Current: {len(original_indices)}, Adding: {needed_count}"
        )

        augmented_images = np.zeros((needed_count, *images[0].shape))
        augmented_labels = np.ones(needed_count, dtype=np.uint8) * class_index

        for i in tqdm(np.arange(needed_count), desc=f"Augmenting class {class_index}"):
            idx = rng.choice(original_indices)
            augmented_images[i] = tensor_to_numpy(
                transform(numpy_to_tensor(images[idx]))
            )

    return augmented_images, augmented_labels


def extend_dataset(
    path: str,
    target_path: str,
    images: np.ndarray,
    labels: np.ndarray,
    overwrite: bool = True,
):

    target_path = Path(target_path)

    if not target_path.is_file():
        shutil.copy(Path(path), target_path)
    elif not overwrite:
        raise FileExistsError(
            "This file already exists. Set 'overwrite = True' to overwrite the file!"
        )

    with h5py.File(target_path, mode="a") as hf:

        images_h5 = hf["images"]
        labels_h5 = hf["ans"]

        original_size = images_h5.shape[0]

        images_h5.resize(original_size + images.shape[0], axis=0)
        labels_h5.resize(original_size + labels.shape[0], axis=0)

        images_h5[original_size:] = images
        labels_h5[original_size:] = labels

In [5]:
with h5py.File("../../.data/Galaxy10_DECals_proc.h5", "r") as hf:
    labels, counts = np.unique(hf["ans"], return_counts=True)

In [ ]:
min_count = int(np.round(np.max(counts) * 1e-2) * 1e2)
for i in tqdm(np.arange(len(counts))):
    label, count = labels[i], counts[i]
    if count < min_count:
        augmented_images, augmented_labels = augmented_class(path="../../.data/Galaxy10_DECals_proc.h5", 
                                                             class_index=label,
                                                             target_count=min_count)
        extend_dataset(
            path="../../.data/Galaxy10_DECals_proc.h5",
            target_path="../../.data/Galaxy10_DECals_augmented.h5",
            images=augmented_images,
            labels=augmented_labels,
            overwrite=True,
        )

        del augmented_images
        del augmented_labels

In [7]:
with h5py.File("../../.data/Galaxy10_DECals_augmented.h5", "r") as hf:
    print(np.unique(hf["ans"], return_counts=True))

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), array([2600, 2600, 2645, 2600, 2600, 2600, 2600, 2628, 2600, 2600]))
